# Rumination classification - averaged participants' epochs

### Imports

In [ ]:
%load_ext lab_black
import os
import pickle
from time import time
import pywt
import mne
import scipy
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium.featurize
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact
from sklearn.decomposition import FastICA
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

import sys

sys.path.append("..")
from utils import *

---------------------------

### Loading data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [ ]:
tmin, tmax = -0.1, 0.6
signal_frequency = 256
ERROR = 0
CORRECT = 1
random_state = 0

In [ ]:
channels_order_list = [
    "Fp1",
    "AF7",
    "AF3",
    "F1",
    "F3",
    "F5",
    "F7",
    "FT7",
    "FC5",
    "FC3",
    "FC1",
    "C1",
    "C3",
    "C5",
    "T7",
    "TP7",
    "CP5",
    "CP3",
    "CP1",
    "P1",
    "P3",
    "P5",
    "P7",
    "P9",
    "PO7",
    "PO3",
    "O1",
    "Iz",
    "Oz",
    "POz",
    "Pz",
    "CPz",
    "Fpz",
    "Fp2",
    "AF8",
    "AF4",
    "AFz",
    "Fz",
    "F2",
    "F4",
    "F6",
    "F8",
    "FT8",
    "FC6",
    "FC4",
    "FC2",
    "FCz",
    "Cz",
    "C2",
    "C4",
    "C6",
    "T8",
    "TP8",
    "CP6",
    "CP4",
    "CP2",
    "P2",
    "P4",
    "P6",
    "P8",
    "P10",
    "PO8",
    "PO4",
    "O2",
]
channels_dict = dict(zip(channels_order_list, np.arange(1, 64, 1)))

Define significant channels - rest will be excluded

In [ ]:
red_box = [
    "F1",
    "Fz",
    "F2",
    "FC1",
    "FCz",
    "FC2",
    "C1",
    "Cz",
    "C2",
    "CP1",
    "CPz",
    "CP2",
    "P1",
    "Pz",
    "P2",
]
significant_channels = [channels_dict[channel] for channel in red_box]

#### Read the data

In [ ]:
df_name = "go_nogo_df_mean"
pickled_data_filename = "../../data/" + df_name + ".pkl"
info_filename = "../../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

#### Average participants' error and correct epochs

In [ ]:
averaged_epochs_df = (
    epochs_df.groupby(
        ["id", "marker"],
        sort=False,
    )
    .apply(
        lambda group_df: pd.Series(
            {
                "epoch": np.mean(group_df["epoch"]),
                "Rumination Full Scale": np.mean(group_df["Rumination Full Scale"]),
            }
        )
    )
    .reset_index()
)

-----------------------

## Training and predictions

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from tempfile import mkdtemp
from sklearn.model_selection import RepeatedKFold


from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")


import numpy as np
import scipy.stats

#### Standard features for EEG analysis provided by Guo et al. (2012)

In [ ]:
def std_signal(t, m, e):
    return np.std(m)


def abs_diffs_signal(t, m, e):
    return np.sum(np.abs(np.diff(m)))


def mean_energy_signal(t, m, e):
    return np.mean(m ** 2)


def skew_signal(t, m, e):
    return scipy.stats.skew(m)


def mean_signal(t, m, e):
    return np.mean(m)

In [ ]:
guo_features = {
    "mean": mean_signal,
    "std": std_signal,
    "mean_energy": mean_energy_signal,
}

step_in_ms = 50
step_tp = int(signal_frequency * step_in_ms / 1000)

#### Calculate p-value with permutation test

In [ ]:
from sklearn.model_selection import permutation_test_score


def calculate_p_permutations(estimator, X, y, cv=3, n_permutations=100, n_jobs=10):

    score_, perm_scores_, pvalue_ = permutation_test_score(
        estimator, X, y, cv=cv, n_permutations=n_permutations, n_jobs=n_jobs
    )

    # summarize
    print(f"     The permutation P-value is = {pvalue_:.3f}")
    #     print(f"     The permutation score is = {score_:.3f}\n")

    return score_, pvalue_

In [ ]:
# from mlxtend.evaluate import paired_ttest_5x2cv
# from sklearn.dummy import DummyClassifier


# def calculate_p(estimator, X, y):
#     baseline_estimator = DummyClassifier(strategy="most_frequent")

#     # check if difference between algorithms is real
#     t, p = paired_ttest_5x2cv(
#         estimator1=baseline_estimator,
#         estimator2=estimator,
#         X=X,
#         y=y,
#         scoring="accuracy",
#         random_seed=0,
#     )

#     # summarize
#     print(f"     The P-value is = {p:.3f}")
#     print(f"     The t-statistics is = {t:.3f}\n")

#     return t, p

#### Validation curves - for parameters' insight

In [ ]:
import matplotlib.pyplot as plt


def pooled_var(stds):
    # https://en.wikipedia.org/wiki/Pooled_variance#Pooled_standard_deviation
    n = 5  # size of each group
    return np.sqrt(sum((n - 1) * (stds ** 2)) / len(stds) * (n - 1))


def show_validation_curves(cv_results, grid_params):

    df = pd.DataFrame(cv_results)
    results = [
        "mean_test_balanced_accuracy",
        "mean_train_balanced_accuracy",
        "std_test_balanced_accuracy",
        "std_train_balanced_accuracy",
    ]

    fig, axes = plt.subplots(
        1, len(grid_params), figsize=(5 * len(grid_params), 7), sharey="row"
    )
    axes[0].set_ylabel("Score", fontsize=25)

    for idx, (param_name, param_range) in enumerate(grid_params.items()):
        grouped_df = df.groupby(f"param_{param_name}")[results].agg(
            {
                "mean_train_balanced_accuracy": "mean",
                "mean_test_balanced_accuracy": "mean",
                "std_train_balanced_accuracy": pooled_var,
                "std_test_balanced_accuracy": pooled_var,
            }
        )

        previous_group = df.groupby(f"param_{param_name}")[results]
        axes[idx].set_xlabel(param_name, fontsize=10)
        axes[idx].set_ylim(0.0, 1.1)
        axes[idx].set_xscale("log")
        lw = 2
        axes[idx].plot(
            param_range,
            grouped_df["mean_train_balanced_accuracy"],
            label="Training score",
            color="darkorange",
            lw=lw,
        )
        axes[idx].fill_between(
            param_range,
            grouped_df["mean_train_balanced_accuracy"]
            - grouped_df["std_train_balanced_accuracy"],
            grouped_df["mean_train_balanced_accuracy"]
            + grouped_df["std_train_balanced_accuracy"],
            alpha=0.2,
            color="darkorange",
            lw=lw,
        )
        axes[idx].plot(
            param_range,
            grouped_df["mean_test_balanced_accuracy"],
            label="Cross-validation score",
            color="navy",
            lw=lw,
        )
        axes[idx].fill_between(
            param_range,
            grouped_df["mean_test_balanced_accuracy"]
            - grouped_df["std_test_balanced_accuracy"],
            grouped_df["mean_test_balanced_accuracy"]
            + grouped_df["std_test_balanced_accuracy"],
            alpha=0.2,
            color="navy",
            lw=lw,
        )

    handles, labels = axes[0].get_legend_handles_labels()
    fig.suptitle("Validation curves", fontsize=40)
    fig.legend(handles, labels, loc=8, ncol=2, fontsize=20)

    fig.subplots_adjust(bottom=0.25, top=0.85)
    plt.show()

----------------------

### Create X and y sets

In [ ]:
dataset = ERROR
dataset_name = "correct" if dataset == CORRECT else "error"

In [ ]:
X_train = np.array(
    averaged_epochs_df[averaged_epochs_df["marker"] == dataset]["epoch"].to_list()
)
y_train = np.array(
    averaged_epochs_df[averaged_epochs_df["marker"] == dataset][
        "Rumination Full Scale"
    ].to_list()
)

Split data by median into two groups: high/low rumination

In [ ]:
rumination_median = np.median(y_train)
HIGH = 1
LOW = 0

In [ ]:
for i in range(len(y_train)):
    if y_train[i] < rumination_median:
        y_train[i] = LOW
    else:
        y_train[i] = HIGH

In [ ]:
X_test = []
y_test = []

In [ ]:
rumination_median

---------------------

### Define data transformers 

In [ ]:
from __future__ import division
from scipy.signal import butter, lfilter
from sklearn.base import TransformerMixin, BaseEstimator


class LowpassFilter(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fs = signal_frequency
        cutoff = 45  # Hz
        B, A = butter(
            6, cutoff / (fs / 2), btype="low", analog=False
        )  # 6th order Butterworth low-pass

        filtered_epochs_per_channel = []
        for channel in X:
            filtered_epochs = np.array(
                [lfilter(B, A, epoch, axis=0) for epoch in channel]
            )
            filtered_epochs_per_channel.append(filtered_epochs)
        filtered_epochs_per_channel = np.array(filtered_epochs_per_channel)
        return filtered_epochs_per_channel


class IcaPreprocessing(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        timepoints_per_channel = np.concatenate(X, axis=1)
        return timepoints_per_channel.T


class IcaPostprocessing(TransformerMixin, BaseEstimator):
    def __init__(self, timepoints_count):
        super().__init__()
        self.timepoints_count = timepoints_count

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ica_transposed = X.T
        ica_n_components = X.shape[1]

        epochs_count = int(X_ica_transposed.shape[1] / self.timepoints_count)
        data_per_channel = X_ica_transposed.reshape(
            ica_n_components, epochs_count, self.timepoints_count
        )

        return data_per_channel


class Cwt(TransformerMixin, BaseEstimator):
    def __init__(self, mwt="morl", cwt_density=2, cwt_octaves=6):
        # for octaves=6, the highest frequency is 45.25 Hz
        super().__init__()
        self.mwt = mwt
        self.cwt_density = cwt_density
        self.cwt_octaves = cwt_octaves

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cwt_per_channel = []
        for data in X:
            data_cwt = np.array(
                [
                    cwt(epoch, self.mwt, self.cwt_density, self.cwt_octaves)
                    for epoch in data
                ]
            )
            cwt_per_channel.append(data_cwt)
        cwt_per_channel = np.array(cwt_per_channel)
        return cwt_per_channel


class CwtFeatureVectorizer(TransformerMixin, BaseEstimator):
    def __init__(self, feature_dict):
        super().__init__()
        self.feature_dict = feature_dict

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectorized_data = []
        for data_cwt in X:
            # cesium functions
            feature_set_cwt = cesium.featurize.featurize_time_series(
                times=None,
                values=data_cwt,
                errors=None,
                features_to_use=list(self.feature_dict.keys()),
                custom_functions=self.feature_dict,
            )
            features_per_epoch = feature_set_cwt.to_numpy()
            vectorized_data.append(features_per_epoch)
        vectorized_data = np.array(vectorized_data)
        return vectorized_data


# reshape data from (channels x epoch x features) to (epochs x channles x features)
# and then flatten it to (epoch x channels*features)
class PostprocessingTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectorized_data = np.stack(X, axis=1)
        epochs_per_channel_feature = vectorized_data.reshape(
            vectorized_data.shape[0], -1
        )
        return epochs_per_channel_feature


class ChannelExtraction(TransformerMixin, BaseEstimator):
    def __init__(self, channel_list):
        super().__init__()
        self.channel_list = channel_list

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        epochs_per_channels = np.transpose(X, (1, 0, 2))
        epochs_per_selected_channels = []

        for channel in self.channel_list:
            this_data = epochs_per_channels[channel]
            epochs_per_selected_channels.append(this_data)

        epochs_per_selected_channels = np.array(epochs_per_selected_channels)
        selected_channels_per_epoch = np.transpose(
            epochs_per_selected_channels, (1, 0, 2)
        )
        #         print(f"EXTRACTION {selected_channels_per_epoch.shape}")
        return selected_channels_per_epoch


# swap channels and epochs axes: from epoch_channel_timepoints to channel_epoch_timepoints and vice versa
class ChannelDataSwap(TransformerMixin, BaseEstimator):
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        data_channel_swaped = np.transpose(X, (1, 0, 2))
        return data_channel_swaped


class BinTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, step):
        super().__init__()
        self.step = step

    def bin_epoch(self, epoch):
        new_channels = []
        for channel in epoch:
            bins_channel = []
            index = 0
            while index + self.step < len(channel):
                this_bin = np.mean(channel[index : index + self.step])
                bins_channel.append(this_bin)
                index += self.step
            new_channels.append(bins_channel)
        return new_channels

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        binned_data = np.array([self.bin_epoch(epoch) for epoch in X])
        return binned_data


# transforms energy of each sub-band into relative energy of sub-band
def RelativeEnergyTransformer():
    def transform(X):
        vectorized_data = []

        for epoch in X:
            total_energy_of_epoch = np.sum(epoch)
            sub_band_relative_energies = np.array(
                [(sub_band_energy / total_energy_of_epoch) for sub_band_energy in epoch]
            )
            vectorized_data.append(sub_band_relative_energies)

        vectorized_data = np.array(vectorized_data)
        return vectorized_data

    return FunctionTransformer(func=transform)

-----------------------
### Define searching experiment

In [ ]:
def rate_classifier(
    X_train, y_train, X_test, y_test, classifier, classifier_params, base_steps, cv=5
):
    # define cross-validation method
    cv_skf = StratifiedKFold(n_splits=3)

    pipeline = Pipeline(steps=base_steps + [classifier])
    param_grid = classifier_params
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=cv_skf,
        scoring={"balanced_accuracy", "precision"},
        refit="balanced_accuracy",
        return_train_score=True,
        n_jobs=10,
        verbose=10,
    )
    grid_search.fit(X_train, y_train)

    return grid_search

In [ ]:
def run_experiment(
    tested_classifiers,
    classifier_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    base_steps,
    results_df,
    function_name="-",
):

    for (classifier, params) in tested_classifiers:
        print(f"Rating {classifier} \n")
        tested_params = {**classifier_params, **params}

        grid_result = rate_classifier(
            X_train,
            y_train,
            X_test,
            y_test,
            classifier,
            tested_params,
            base_steps,
            cv=2,
        )

        #     predictions = grid_result.predict(X_test)
        #     r2 = grid_result.score(X_test, y_test)
        #     mae = mean_absolute_error(y_test, predictions)
        #     r2_adj = r2_adjusted_scorer(y_test, predictions, len(X_test[0]), len(X_test))

        best_estimator_index = grid_result.best_index_
        mean_cv_balanced_accuracy = grid_result.cv_results_[
            "mean_test_balanced_accuracy"
        ][best_estimator_index]
        std_cv_balanced_accuracy = grid_result.cv_results_[
            "std_test_balanced_accuracy"
        ][best_estimator_index]
        mean_cv_precision = grid_result.cv_results_["mean_test_precision"][
            best_estimator_index
        ]
        std_cv_precision = grid_result.cv_results_["std_test_precision"][
            best_estimator_index
        ]
        mean_train_balanced_accuracy = grid_result.cv_results_[
            "mean_train_balanced_accuracy"
        ][best_estimator_index]

        print(f"     Best parameters: {grid_result.best_params_}")
        print(
            f"     mean acc: {mean_cv_balanced_accuracy}           ± {round(std_cv_balanced_accuracy,3)}"
        )
        print(f"     mean acc train: {mean_train_balanced_accuracy}")

        cv_results = grid_result.cv_results_
        #         t_statistics, p_value = calculate_p(
        #             grid_result.best_estimator_, X_train, y_train
        #         )

        scores_, pvalue_ = calculate_p_permutations(
            grid_result.best_estimator_, X_train, y_train
        )

        #         show_validation_curves(grid_result.cv_results_, tested_params)

        data = {
            "data_set": dataset_name,
            "pipeline_name": pipeline_name,
            "model": classifier[0],
            "parameters": grid_result.best_params_,
            "mean_cv_balanced_accuracy": mean_cv_balanced_accuracy,
            "std_cv_balanced_accuracy": std_cv_balanced_accuracy,
            "mean_cv_precision": mean_cv_precision,
            "std_cv_precision": std_cv_precision,
            "cv_results": cv_results,
            "mean_train_balanced_accuracy": mean_train_balanced_accuracy,
            "p-value": pvalue_,
            "best_estimator": grid_result.best_estimator_,
            #             "t-stats": t_statistics,
        }

        results_df = results_df.append(data, ignore_index=True)
    return results_df

-----------------
### Define architectures

In [ ]:
# ERP-bins + ERP-bins-cwt-features


def erp_bins_features_steps(feature_function_dict):

    functions_base_steps = [
        ("cwt", Cwt()),
        (
            "cwt_feature",
            CwtFeatureVectorizer(feature_dict=feature_function_dict),
        ),
        ("postprocessing_func", PostprocessingTransformer()),
    ]
    functions_pipeline = Pipeline(steps=functions_base_steps)

    bins_base_steps = [
        ("data_channel_swap_after_filter", ChannelDataSwap()),
        ("binning", BinTransformer(step=step_tp)),
        ("data_channel_swap", ChannelDataSwap()),
        ("postprocessing_bins", PostprocessingTransformer()),
    ]
    bins_pipeline = Pipeline(steps=bins_base_steps)

    combined_features = FeatureUnion(
        [("bins", bins_pipeline), ("functins", functions_pipeline)]
    )

    steps = [
        (
            "channels_filtering",
            ChannelExtraction(significant_channels),
        ),
        ("data_channel_swap_filter", ChannelDataSwap()),
        ("lowpass_filter", LowpassFilter()),
        ("features", combined_features),
        ("scaler", StandardScaler()),
        ("feature_selection", PCA(random_state=random_state)),
    ]

    return steps

In [ ]:
# Erp-bins
def erp_bins_steps():
    steps = [
        (
            "channels_filtering",
            ChannelExtraction(significant_channels),
        ),
        ("data_channel_swap_filter", ChannelDataSwap()),
        ("lowpass_filter", LowpassFilter()),
        ("data_channel_swap_after_filter", ChannelDataSwap()),
        ("binning", BinTransformer(step=step_tp)),
        ("data_channel_swap", ChannelDataSwap()),
        ("postprocessing", PostprocessingTransformer()),
        ("scaler", StandardScaler()),
        ("feature_selection", PCA(random_state=random_state)),
    ]

    return steps

In [ ]:
# ICA-bins + ICA-bins-cwt-features


def ica_bins_features_steps(feature_function_dict):

    steps = [
        (
            "channels_filtering",
            ChannelExtraction(significant_channels),
        ),
        ("ica_preprocessing", IcaPreprocessing()),
        #         ("ica", FastICA(random_state=random_state)),
        ("spatial_filter", PCA(random_state=random_state)),
        (
            "ica_postprocessing",
            IcaPostprocessing(timepoints_count=X_train.shape[-1]),
        ),
        ("lowpass_filter", LowpassFilter()),
        (
            "features",
            FeatureUnion(
                [
                    (
                        "bins",
                        Pipeline(
                            [
                                ("channel_data_swap", ChannelDataSwap()),
                                ("binning", BinTransformer(step=step_tp)),
                                ("data_channel_swap", ChannelDataSwap()),
                                ("postprocessing_bins", PostprocessingTransformer()),
                            ]
                        ),
                    ),
                    (
                        "functions",
                        Pipeline(
                            [
                                ("cwt", Cwt()),
                                (
                                    "cwt_feature",
                                    CwtFeatureVectorizer(
                                        feature_dict=feature_function_dict
                                    ),
                                ),
                                (
                                    "postprocessing_functions",
                                    PostprocessingTransformer(),
                                ),
                            ]
                        ),
                    ),
                ]
            ),
        ),
        ("scaler", StandardScaler()),
        ("feature_selection", PCA(random_state=random_state)),
    ]

    return steps

In [ ]:
# ICA-bins


def ica_bins_steps():

    steps = [
        (
            "channels_filtering",
            ChannelExtraction(significant_channels),
        ),
        ("ica_preprocessing", IcaPreprocessing()),
        #         ("ica", FastICA(random_state=random_state)),
        ("spatial_filter", PCA(random_state=random_state)),
        (
            "ica_postprocessing",
            IcaPostprocessing(timepoints_count=X_train.shape[-1]),
        ),
        ("lowpass_filter", LowpassFilter()),
        ("channel_data_swap", ChannelDataSwap()),
        ("binning", BinTransformer(step=step_tp)),
        ("data_channel_swap", ChannelDataSwap()),
        ("postprocessing", PostprocessingTransformer()),
        ("scaler", StandardScaler()),
        ("feature_selection", PCA(random_state=random_state)),
    ]

    return steps

# Experiments

In [ ]:
classifier_params = dict(
    spatial_filter__n_components=np.arange(1, 15, 2),
    feature_selection__n_components=np.arange(3, 9, 1),
)

### Searching best regularization parameters

In [ ]:
res = pd.DataFrame()

In [ ]:
# Done
# pipeline_name = "ICA_bins_cwt_features"
# pipeline_name='PCA_15_bins_functions'
# this_steps = ica_bins_features_steps(guo_features)

In [ ]:
# # Done
# pipeline_name = "ERP_bins_cwt_features"
# this_steps = erp_bins_features_steps(guo_features)

In [ ]:
# Done
pipeline_name = "PCA_15_bins"
# pipeline_name = "PCA_4_bins"
this_steps = ica_bins_steps()

In [ ]:
# # Done
# pipeline_name = "ERP_bins"
# this_steps = erp_bins_steps()

In [ ]:
# the best the best - c=10, gamma=0.001 - ICA_bins
for C in [0.0001, 0.001, 0.01, 0.1, 1, 10]:
    #     for epsilon in [0.0001, 0.001, 0.01, 0.1, 1]:
    for gamma in [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]:

        svc = ("svc", SVC())
        svc_params = dict(
            svc__kernel=["rbf"],
            svc__C=[C],
            svc__gamma=[gamma],
            #             svc__epsilon=[epsilon],
        )

        tested_classifiers = [
            (svc, svc_params),
        ]

        # rate different models
        res = run_experiment(
            tested_classifiers,
            classifier_params,
            pipeline_name,
            X_train,
            X_test,
            y_train,
            y_test,
            dataset_name,
            this_steps,
            res,
        )

In [ ]:
for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
    #     for epsilon in [0.0001, 0.001, 0.01, 0.1, 1]:
    #         for gamma in [0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1]:

    svc = ("svc", SVC())
    svc_params = dict(
        svc__kernel=["linear"],
        svc__C=[C],
        svc__gamma=["scale"],
        #             svc__epsilon=[epsilon],
    )

    tested_classifiers = [
        (svc, svc_params),
    ]

    # rate different models
    res = run_experiment(
        tested_classifiers,
        classifier_params,
        pipeline_name,
        X_train,
        X_test,
        y_train,
        y_test,
        dataset_name,
        this_steps,
        res,
    )

In [ ]:
# C=0.01 l1=0.01 , 0.1,0.001,0.1,0.5

for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
    for l1_ratio in [
        0.0000001,
        0.000001,
        0.00001,
        0.0001,
        0.001,
        0.01,
        0.1,
        0.3,
        0.5,
        0.7,
        1,
    ]:

        lr = ("lr", LogisticRegression())
        lr_params = dict(
            lr__penalty=["elasticnet"],
            lr__solver=["saga"],
            lr__l1_ratio=[l1_ratio],
            lr__C=[C],
            lr__random_state=[random_state],
        )

        tested_classifiers = [(lr, lr_params)]

        # rate different models
        res = run_experiment(
            tested_classifiers,
            classifier_params,
            pipeline_name,
            X_train,
            X_test,
            y_train,
            y_test,
            dataset_name,
            this_steps,
            res,
        )

In [ ]:
# for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:

#     lr = ("lr", LogisticRegression())
#     lr_params = dict(lr__C=[C])

#     tested_classifiers = [(lr, lr_params)]

#     # rate different models
#     results_df = run_experiment(
#         tested_classifiers,
#         classifier_params,
#         pipeline_name,
#         X_train,
#         X_test,
#         y_train,
#         y_test,
#         dataset_name,
#         this_steps,
#         results_df,
#     )

#### Run Grid Search 

In [ ]:
results_df = pd.DataFrame()

In [ ]:
svc_rbf = ("svc_rbf", SVC())
svc_rbf_params = dict(
    svc_rbf__kernel=["rbf"],
    svc_rbf__C=[1],
    svc_rbf__gamma=[0.0000001],
)

svc_lin = ("svc_lin", SVC())
svc_lin_params = dict(
    svc_lin__kernel=["linear"],
    svc_lin__C=[0.01],
    svc_lin__gamma=["scale"],
)


lr_en = ("lr_en", LogisticRegression())
lr_en_params = dict(
    lr_en__penalty=["elasticnet"],
    lr_en__solver=["saga"],
    lr_en__C=[0.01],
    lr_en__l1_ratio=[0.000001],
    lr_en__random_state=[random_state],
)


lr = ("lr", LogisticRegression())
lr_params = dict(lr__C=[0.01])


tested_classifiers = [
    (svc_rbf, svc_rbf_params),
    (svc_lin, svc_lin_params),
    (lr_en, lr_en_params),
    #     (lr, lr_params),
]

In [ ]:
# rate different models
results_df = run_experiment(
    tested_classifiers,
    classifier_params,
    pipeline_name,
    X_train,
    X_test,
    y_train,
    y_test,
    dataset_name,
    this_steps,
    results_df,
)

In [ ]:
# with pd.option_context("display.max_rows", 2000):
#     display(results_df)

#### Save the data

In [ ]:
results_df.to_pickle("../../data/classification_PCA_" + dataset_name + ".pkl")

In [ ]:
results_viz = results_df.drop(columns=["best_estimator"])
results_viz.to_pickle(
    "../../data/classification_PCA_vizualization_" + dataset_name + ".pkl"
)

-------------------

#### Recalculate p-values with permutation test

In [ ]:
file_name_ICA = "../../data/results_classification/classification_ICA_error.pkl"
results_ICA = pd.read_pickle(file_name_ICA)

file_name_PCA = "../../data/results_classification/classification_PCA_error.pkl"
results_PCA = pd.read_pickle(file_name_PCA)

file_name_ERP = "../../data/results_classification/classification_ERP_error.pkl"
results_ERP = pd.read_pickle(file_name_ERP)

In [ ]:
results_df = pd.concat([results_ICA, results_PCA, results_ERP], ignore_index=True)

In [ ]:
results_without_func_df = results_df[
    results_df["pipeline_name"].isin(
        ["ERP_bins", "PCA_15_bins", "PCA_4_bins", "ICA_15_bins", "ICA_4_bins"]
    )
]

In [ ]:
permutation_results = []

for index, row in results_without_func_df.iterrows():
    estimator = row.best_estimator
    current_acc = row.mean_cv_balanced_accuracy
    #     print(estimator)
    cv_skf = StratifiedKFold(n_splits=3)

    score_, perm_scores_, pvalue_ = permutation_test_score(
        estimator,
        X_train,
        y_train,
        scoring="balanced_accuracy",
        cv=cv_skf,
        n_permutations=1000,
        n_jobs=11,
    )
    print(f"Score: {score_}   df_score: {current_acc}  p_value: {pvalue_}  ")
    permutation_results.append((score_, perm_scores_, pvalue_))

In [ ]:
permutation_results = np.array(permutation_results)
np.save("permutation_results.npy", permutation_results, allow_pickle=True)
perm_res = np.load("permutation_results.npy", allow_pickle=True)

In [ ]:
b = np.split(perm_res, indices_or_sections=3, axis=1)
scores = b[0]
perm_scores = b[1]
p_val = b[2]

In [ ]:
results_without_func_df_copy = results_without_func_df.copy()

In [ ]:
perm_scores_df = pd.DataFrame(perm_scores)

In [ ]:
p_values_df = pd.DataFrame(p_val)

In [ ]:
results_without_func_df["p-value"] = p_values_df
results_without_func_df["permutation_score"] = perm_scores_df

In [ ]:
results_without_func_df

In [ ]:
results_without_func_df.to_pickle(
    "classification_all_results_without_functions_" + dataset_name + ".pkl"
)
results_without_func_df_viz = results_without_func_df.drop(columns=["best_estimator"])
results_without_func_df_viz.to_pickle(
    "classification_all_results_without_functions_viz" + dataset_name + ".pkl"
)
results_without_func_df_viz.to_csv(
    "classification_all_results_without_functions_viz" + dataset_name + ".csv"
)